In [2]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime
import scipy as sp
import time
from datetime import datetime, date, timedelta
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from dateutil.relativedelta import relativedelta

import io
from io import StringIO 
#import datedelta
import calendar
import  csv
import json
import random

import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from scipy import stats


def add_delta_cols(df):
    #df['delta_vector'] = 0
    #df['delta_max_vector'] = 0

    df['delta_vector'] = 0
    df['delta_max'] = 0
    df['upper_wick'] = 0
    df['lower_wick'] = 0
    
    dv_col = df.columns.get_loc('delta_vector')
    dm_col = df.columns.get_loc('delta_max')
    uw_col = df.columns.get_loc('upper_wick')
    lw_col = df.columns.get_loc('lower_wick')
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    c_col = df.columns.get_loc('c')
    arr = df.values
    for i in range(arr.shape[0]):
        arr[i,dv_col] = arr[i,c_col] - arr[i,o_col]
        arr[i,dm_col] = arr[i,h_col] - arr[i,l_col]
        if arr[i,c_col] > arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,c_col]
            arr[i,lw_col] = arr[i,o_col] - arr[i,l_col]
            
        elif arr[i,c_col] < arr[i,o_col]:
            arr[i,uw_col] = arr[i,h_col] - arr[i,o_col]
            arr[i,lw_col] = arr[i,c_col] - arr[i,l_col]   
            
    df = pd.DataFrame(arr,columns = df.columns)
    df['delta'] = abs(df['delta_vector'])
    
    return df
def load_df(pair = 'EUR_USD'
            ,granularity = 'M5'
            ,start = datetime(2016,1,1,0,0,0)
            ,end = datetime(2022,7,31,0,0,0)):
    def convert_timestamp(df):
        time_col = df.columns.get_loc('time')
        df['minute_col'] = 0
        df['hour_col'] = 0
        new_col = df.columns.get_loc('minute_col')
        new_col2 = df.columns.get_loc('hour_col')
        arr = df.values 
        for i in range(arr.shape[0]):
            arr[i,new_col] = pd.Timestamp(arr[i,time_col]).minute
            arr[i,new_col2] = pd.Timestamp(arr[i,time_col]).hour
        return pd.DataFrame(arr,columns = df.columns)
    dir_name = os.getcwd() + '/' + pair + '_' + granularity
    path = dir_name + '/' + str(date(start.year,start.month,start.day)) + '_' + str(date(end.year,end.month,end.day)) + '.csv'
    print('PATH:',path)
    df = pd.read_csv(path)
    df = add_delta_cols(df)
    df = convert_timestamp(df)
    df = df.drop_duplicates(subset=['utc_timestamp'])
    print(df.shape)
    return df



def get_support(df,
                df2,
                lookup_range = 1500,
                pip_delta = .001,
                time_range = 11,
                high_spread = .0005,
                low_spread = 0,
                percentile = 98
               ):
    print('GETTING SUPPORT INDICATOR')
    print('LOOKUP RANGE AND PIP DELTA:',lookup_range,pip_delta)
    s = time.time()
    c_col = df.columns.get_loc('c')
    o_col = df.columns.get_loc('o')
    l_col = df.columns.get_loc('l')
    h_col = df.columns.get_loc('h')
    d_col = df.columns.get_loc('delta')
    minute_col = df.columns.get_loc('minute_col')
    df['support_indicator'] = 0
    new_col = df.columns.get_loc('support_indicator')
    arr = df.values
    min_count = 0
    c = 0 
   # high_spread = .0003
    #low_spread = .0001
    #low_spread = 0
    #high_spread = .0005
    min_filter = np.percentile(df2['delta'],percentile)
    
    for i in range(max(lookup_range,time_range + 1),arr.shape[0]):
       # try:
        #DEFINE SUPPORT/RESISTANCE as the MIN or MAX of a lookup range

        if (arr[i - time_range,o_col] - arr[i,c_col]) >= min_filter and arr[i,minute_col] == 55 and sum(arr[i - 50: i,new_col]) == 0:
            arr[i,new_col] = 1

    print('COUNT:',c)

    e = time.time()
    print('TOTAL FUNCTION TIME:',(e-s)/60,' MINUTES')
    df = pd.DataFrame(arr,columns = df.columns)
    print('SUPPORT INDICATOR SHAPE',df[df['support_indicator'] == 1].shape)
    return df
def test_future(df,pair,column = 'support_indicator'):
    if 'JPY' in pair:
        mult = 100
    else:
        mult = 100 * 100
    c_col = df.columns.get_loc('c')
    col = df.columns.get_loc(column)
    
    lst = []
    arr = df.values
    for i in range(arr.shape[0] - 101):
        if arr[i,col] == 1:
            c = arr[i,c_col]

            lst.append([mult * (arr[i + 6,c_col] - c),
                        mult * (arr[i + 12,c_col] - c),
                        mult * (arr[i + 24,c_col] - c),
                        mult * (arr[i + 48,c_col] - c),
                        mult * (arr[i + 100,c_col] - c)                       
                       ])
    df2 = pd.DataFrame(lst,columns = ['next_30','next_hour','next_two_hours','next_four_hours','next_eight_hours'])  
    print('MEDIAN:')
    print(df2.median())
    print()
    print('MEAN:')
    print(df2.mean())
    print()
    print('SUM:')
    print(print(pd.DataFrame(df2.sum()).T))
    
    return pd.DataFrame(df2.median(),columns = [pair]).T,pd.DataFrame(df2.mean(),columns = [pair]).T,pd.DataFrame(df2.sum(),columns = [pair]).T



In [ ]:
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score, roc_auc_score, roc_curve,recall_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
SEED = 314159265
VALID_SIZE = 0.2
TARGET = 'outcome'
def save_xgb_model(model,params,auc_score,auc_score2):
    print('SAVING XGB MODEL')
    import json
    project_folder = 'risk_ml_model'
    prefix = os.getcwd() + '/ML_MODEL_V1/AUC_train_' + str(round(auc_score2 * 100,2)) + '_AUC_test_' + str(round(auc_score * 100,2))
    print('saving xgb model:',prefix)
    directory = prefix
    isdir = os.path.isdir(directory) 
    if isdir:
        print('DIRECTORY:',directory,' EXISTS, passing...')
        pass
    else:
        print('MAKING DIRECTORY:',directory)
        os.mkdir(directory) 
    file_name = prefix + '/xgb.pkl'

    # save
    pickle.dump(model, open(file_name, "wb"))
    

    
    # open file for writing
    f = open(prefix + '/params.json',"w")

    # write file
    f.write( str(params) )

    # close file
    f.close()




def score(params):
    print("Training with params:")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    
    dtrain = xgb.DMatrix(X_train, label=y_train, feature_types=feature_types)
    dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_types=feature_types)
    
    
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round, evals=watchlist, verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = roc_auc_score(y_valid, predictions)    
    # Predictions for validation and train sets
    y_pred_valid = gbm_model.predict(dvalid, ntree_limit=gbm_model.best_iteration + 1)
    y_pred_train = gbm_model.predict(dtrain, ntree_limit=gbm_model.best_iteration + 1)
    
    # Calculate metrics for validation set
    cm_valid = confusion_matrix(y_valid, np.round(y_pred_valid))
    f1_valid = f1_score(y_valid, np.round(y_pred_valid))
    precision_valid = precision_score(y_valid, np.round(y_pred_valid))
    recall_valid = recall_score(y_valid, np.round(y_pred_valid))
    accuracy_valid = accuracy_score(y_valid, np.round(y_pred_valid))
    auc_valid = roc_auc_score(y_valid, y_pred_valid)
    
    # Calculate metrics for train set
    cm_train = confusion_matrix(y_train, np.round(y_pred_train))
    f1_train = f1_score(y_train, np.round(y_pred_train))
    precision_train = precision_score(y_train, np.round(y_pred_train))
    recall_train = recall_score(y_train, np.round(y_pred_train))
    accuracy_train = accuracy_score(y_train, np.round(y_pred_train))
    auc_train = roc_auc_score(y_train, y_pred_train)
    
    # Print metrics
    print("\nValidation set metrics:")
    print(f"Confusion matrix:\n{cm_valid}")
    print(f"F1 score: {f1_valid}")
    print(f"Precision: {precision_valid}")
    print(f"Recall: {recall_valid}")
    print(f"Accuracy: {accuracy_valid}")
    print(f"AUC score: {auc_valid}")
    
    print("\nTrain set metrics:")
    print(f"Confusion matrix:\n{cm_train}")
    print(f"F1 score: {f1_train}")
    print(f"Precision: {precision_train}")
    print(f"Recall: {recall_train}")
    print(f"Accuracy: {accuracy_train}")
    print(f"AUC score: {auc_train}")

    # Plot AUC chart for both validation and train sets
    fpr_valid, tpr_valid, _ = roc_curve(y_valid, y_pred_valid)
    fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_train)
    
    plt.figure()
    plt.plot(fpr_valid, tpr_valid, label=f'Validation AUC = {auc_valid:.2f}')
    plt.plot(fpr_train, tpr_train, label=f'Train AUC = {auc_train:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc='lower right')
    plt.show()
    
    
    plot_importance(gbm_model, title='Feature importances', xlabel='F score', ylabel='Features', importance_type='weight')
    #plt.show()

   # save_xgb_model(model=gbm_model, params=params, auc_score=auc_valid)
    save_xgb_model(model = gbm_model,params = params,auc_score = score,auc_score2 = auc_train)
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

def optimize(
             trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 50, 250, 1),
         #'n_estimators': 10,
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        #'tree_method': 'exact',
        'tree_method': 'hist',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                max_evals=5, trials=trials)
    return best
def load_train_and_test_data(start,end,sl,tp,bullish,spread):


    file = 'forex_ml_training_data_' + str(start.date()) + '_' + str(end.date()) + '_TP_' + str(tp) + '_SL_' + str(sl) + '_BULLISH_' + str(bullish) + '_SPREAD_' + str(spread) + '.csv'
    file2 = 'forex_ml_validation_data_' + str(start.date()) + '_' + str(end.date()) + '_TP_' + str(tp) + '_SL_' + str(sl) + '_BULLISH_' + str(bullish) + '_SPREAD_' + str(spread) + '.csv'
    file3 = 'forex_ml_testing_data_' + str(start.date()) + '_' + str(end.date()) + '_TP_' + str(tp) + '_SL_' + str(sl) + '_BULLISH_' + str(bullish) + '_SPREAD_' + str(spread) + '.csv'
    
    
    cols = ['pair','bearish_engulfing', 'bullish_engulfing', 'bearish_tls', 'bullish_tls',
           'previous_hour_volume', 'previous_4_hour_volume',
           'previous_bullish_hour', 'previous_bullish_4hour',
           'previous_bullish_8hour', 'previous_bullish_24hour',
           'previous_bullish_50hour', 'previous_bullish_120hour',
           'previous_bullish_250hour', 'previous_bullish_400hour',
           'previous_delta_hour', 'previous_delta_4hour', 'previous_delta_8hour',
           'previous_delta_24hour', 'previous_delta_50hour',
           'previous_delta_120hour', 'previous_delta_250hour',
           'previous_delta_400hour', 'previous_lower_wick_hour',
           'previous_upper_wick_hour', 'low_at_low_of_last_day',
           'low_at_low_of_last_5day', 'low_at_low_of_last_10day',
           'low_at_low_of_last_20day', 'high_at_high_of_last_day',
           'high_at_high_of_last_5day', 'high_at_high_of_last_10day',
           'high_at_high_of_last_20day',
           'close_difference_from_low_close_of_last_day',
           'close_difference_from_low_close_of_last_5day',
           'close_difference_from_low_close_of_last_10day',
           'close_difference_from_high_close_of_last_day',
           'close_difference_from_high_close_of_last_5day',
           'close_difference_from_high_close_of_last_10day', 'hour_0_3',
           'hour_4_7', 'hour_8_11', 'hour_12_15', 'hour_16_19', 'hour_20_23',
           'previous_5_min_volume', 'previous_15_min_volume', 'bullish_ema_5min',
           'bearish_ema_5min', 'previous_bullish_5min', 'previous_delta_5min',
           'bearish_engulfing5', 'bullish_engulfing5', 'bearish_tls5']


    df = pd.read_csv(file)
    df = df.sample(frac=1).reset_index(drop=True)
    print(df.shape)
    df.head()
    X = df.drop(columns = ['target'])
    X = X[cols]
    y = df['target']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    print(X_train.shape,X_valid.shape)


    df = pd.read_csv(file2)
    df = df.sample(frac=1).reset_index(drop=True)
    print(df.shape)
    df.head()
    X = df.drop(columns = ['target'])
    X_test = X[cols]
    y_test = df['target']
    
    

    print(X_train.shape,X_test.shape)

    df = pd.read_csv(file3)
    df = df.sample(frac=1).reset_index(drop=True)
    print(df.shape)
    X = df.drop(columns = ['target'])
    X_test2 = X[cols]
    y_test2 = df['target']    
    
    
    X_train = pd.concat([X_train,X_test.iloc[:250000,:]])
    y_train = pd.concat([y_train,y_test.iloc[:250000]])
    X_test = X_test.iloc[250000:,:]
    y_test = y_test.iloc[250000:]
    
    print('SHAPES:',X_train.shape, X_valid.shape,X_test.shape, X_test2.shape)
    
    
    
    return X_train, X_valid, y_train, y_valid,X_test, X_test2, y_test, y_test2


def get_feature_types(df):
    feature_types = []
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.integer):
            feature_types.append('int')
        elif np.issubdtype(df[col].dtype, np.floating):
            feature_types.append('float')
        elif df[col].dtype == 'object':
            feature_types.append('c')
        else:
            feature_types.append('unknown')
    return feature_types

start = datetime(2005,1,1,0,0,0)
end = datetime(2023,1,1,0,0,0)


sl = 5
tp = 100
bullish = True
spread = .5

X_train, X_valid, y_train, y_valid,X_test, X_test2, y_test, y_test2 = load_train_and_test_data(start,end,sl,tp,bullish,spread)

encoder = LabelEncoder()
feature_types = get_feature_types(X_train)
col = 'pair'
encoder.fit(X_train[col])
X_train[col] = encoder.transform(X_train[col])
X_valid[col] = encoder.transform(X_valid[col])
X_test[col] = encoder.transform(X_test[col])
X_test2[col] = encoder.transform(X_test2[col])

X_train.head()




In [1]:
!pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Run the optimization
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix
import os
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import xgboost as xgb
                        
def print_results(gbm_model,dvalid,y_valid,label,threshold = 0.65):
    predictions_prob = gbm_model.predict(dvalid)

    # Set a threshold for classifying examples into the positive class
    
    predictions = (predictions_prob > threshold).astype(int)

    # Calculate the AUC score
    cm = confusion_matrix(y_valid, predictions)
    num_trades = int(cm[1][1] + cm[0][1])
    if num_trades == 0:
        pass
    else:    
        auc = roc_auc_score(y_valid, predictions_prob)

        # Calculate the precision and recall scores
        precision = precision_score(y_valid, predictions)
        recall = recall_score(y_valid, predictions)

    # Calculate the confusion matrix
    

    return cm
def run_simulator(gbm_model,dvalid,y_valid,rr,train,threshold):
   # dtest = xgb.DMatrix(X_valid, label=y_valid)  
    min_ = 1
    max_ = 99
    balance = 0
    best_threshold = -1
    if train:
        for threshold in range(min_,max_):
            threshold = threshold / 100
            cm = print_results(gbm_model,dvalid,y_valid,label = 'TEST',threshold = threshold)
            num_trades = int(cm[1][1] + cm[0][1])
            if num_trades == 0:
                pass
            else:
                win_rate = cm[1][1]/(cm[1][1] + cm[0][1])
                loss_rate = 1 - win_rate

                ev = (win_rate * rr) - (loss_rate * 1) 
                bal = 10000
                for i in range(num_trades):
                    bal += (bal * (ev/100))
                    bal -= (bal * .003) #spread and commission cost
                if bal > balance:
                    balance = bal
                    best_threshold = threshold
                if balance > 10000000:
                    balance = 10000000
    else:
        cm = print_results(gbm_model,dvalid,y_valid,label = 'TEST',threshold = threshold)
        num_trades = int(cm[1][1] + cm[0][1])
        if num_trades == 0:
            pass
        else:
            win_rate = cm[1][1]/(cm[1][1] + cm[0][1])
            loss_rate = 1 - win_rate

            ev = (win_rate * rr) - (loss_rate * 1) 
            bal = 10000
            for i in range(num_trades):
                bal += (bal * (ev/100))
                bal -= (bal * .003) #spread and commission cost
            if bal > balance:
                balance = bal
                best_threshold = threshold
            if balance > 10000000:
                balance = 10000000
    balance = round(balance)
    print('final balance',balance,'threshold',best_threshold)
    return balance,best_threshold

def save_xgb_model(model,params,auc_score,auc_score2,train_balance,valid_balance,test_balance,balance_2022,threshold):
    print('SAVING XGB MODEL')
    import json
    
    #directory_path = os.getcwd() + "/ML_MODEL_V3_BULLISH_" + str(bullish) + "_TP_" + str(tp) + "_SL_" + str(sl) + "/"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    prefix = directory_path + 'AUC_train_' + str(round(auc_score2 * 100,2)) + \
        '_AUC_test_' + str(round(auc_score * 100,2)) + '_SIM_BALANCE_TRAIN_' + str(train_balance) \
    + '_VALID_' + str(valid_balance)+ '_TEST_' + str(test_balance) + '_2022_' + str(balance_2022) + '_threshold_' + str(round(threshold,3))
    print('saving xgb model:',prefix)
    directory = prefix
    isdir = os.path.isdir(directory) 
    if isdir:
        print('DIRECTORY:',directory,' EXISTS, passing...')
        pass
    else:
        print('MAKING DIRECTORY:',directory)
        os.mkdir(directory) 
    file_name = prefix + '/xgb.pkl'

    # save
    pickle.dump(model, open(file_name, "wb"))
    

    
    # open file for writing
    f = open(prefix + '/params.json',"w")

    # write file
    f.write( str(params) )

    # close file
    f.close()




def score(params):
    print("Training with params:")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    
    dtrain = xgb.DMatrix(X_train, label=y_train, feature_types=feature_types)
    dvalid = xgb.DMatrix(X_valid, label=y_valid, feature_types=feature_types)
    dtest = xgb.DMatrix(X_test, label=y_test, feature_types=feature_types)
    dtest2 = xgb.DMatrix(X_test2, label=y_test2, feature_types=feature_types)
    
    #dtrain = xgb.DMatrix(X_train, label=y_train)
    #dvalid = xgb.DMatrix(X_valid, label=y_valid)    
    
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round, evals=watchlist,early_stopping_rounds=5, verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = roc_auc_score(y_valid, predictions)    
    # Predictions for validation and train sets
    y_pred_valid = gbm_model.predict(dvalid, ntree_limit=gbm_model.best_iteration + 1)
    y_pred_train = gbm_model.predict(dtrain, ntree_limit=gbm_model.best_iteration + 1)
    
    # Calculate metrics for validation set
    cm_valid = confusion_matrix(y_valid, np.round(y_pred_valid))
    f1_valid = f1_score(y_valid, np.round(y_pred_valid))
    precision_valid = precision_score(y_valid, np.round(y_pred_valid))
    recall_valid = recall_score(y_valid, np.round(y_pred_valid))
    accuracy_valid = accuracy_score(y_valid, np.round(y_pred_valid))
    auc_valid = roc_auc_score(y_valid, y_pred_valid)
    
    # Calculate metrics for train set
    cm_train = confusion_matrix(y_train, np.round(y_pred_train))
    f1_train = f1_score(y_train, np.round(y_pred_train))
    precision_train = precision_score(y_train, np.round(y_pred_train))
    recall_train = recall_score(y_train, np.round(y_pred_train))
    accuracy_train = accuracy_score(y_train, np.round(y_pred_train))
    auc_train = roc_auc_score(y_train, y_pred_train)
    
    
    y_pred_test = gbm_model.predict(dtest, ntree_limit=gbm_model.best_iteration + 1)
    auc_test = roc_auc_score(y_test, y_pred_test)
    
    # Print metrics
    print("\nValidation set metrics:")
    print(f"Confusion matrix:\n{cm_valid}")
    print(f"F1 score: {f1_valid}")
    print(f"Precision: {precision_valid}")
    print(f"Recall: {recall_valid}")
    print(f"Accuracy: {accuracy_valid}")
    print(f"AUC score: {auc_valid}")
    
    print("\nTrain set metrics:")
    print(f"Confusion matrix:\n{cm_train}")
    print(f"F1 score: {f1_train}")
    print(f"Precision: {precision_train}")
    print(f"Recall: {recall_train}")
    print(f"Accuracy: {accuracy_train}")
    print(f"AUC score: {auc_train}")

    # Plot AUC chart for both validation and train sets
    fpr_valid, tpr_valid, _ = roc_curve(y_valid, y_pred_valid)
    fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_train)
    
    plt.figure()
    plt.plot(fpr_valid, tpr_valid, label=f'Validation AUC = {auc_valid:.2f}')
    plt.plot(fpr_train, tpr_train, label=f'Train AUC = {auc_train:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc='lower right')
    plt.show()
    
    
    plot_importance(gbm_model, title='Feature importances', xlabel='F score', ylabel='Features', importance_type='weight')
    #plt.show()
    
    
    midpoint = 0
    
    if midpoint != 0:
        print('RUNNING SIM ON TRAIN SET')
        train_balance,threshold =  run_simulator(gbm_model,dvalid = dtrain,y_valid = y_train,rr = tp / sl,train = True,threshold = 0)

        print('RUNNING SIM ON TEST SET')
        test_balance,threshold3 =  run_simulator(gbm_model,dvalid = dtest,y_valid = y_test,rr = tp / sl,train = True,threshold = 0)

        threshold_test = threshold + .05
        print("2 Thresholds",threshold,threshold3,' MIDPOINT',(threshold + threshold3) / 2)
        threshold = (threshold + threshold3) / 2

        print('RUNNING SIM ON VALID SET')
        valid_balance,threshold_ =  run_simulator(gbm_model,dvalid = dvalid,y_valid = y_valid,rr = tp / sl,train = False,threshold = threshold)

        print('RUNNING SIM ON 2022 SET')
        balance_2022,threshold_ =  run_simulator(gbm_model,dvalid = dtest2,y_valid = y_test2,rr = tp / sl,train = False,threshold = threshold)

    else:
        print('RUNNING SIM ON TRAIN SET')
        train_balance,threshold =  run_simulator(gbm_model,dvalid = dtrain,y_valid = y_train,rr = tp / sl,train = True,threshold = 0)

        print('RUNNING SIM ON TEST SET')
        test_balance,threshold3 =  run_simulator(gbm_model,dvalid = dtest,y_valid = y_test,rr = tp / sl,train = False,threshold = threshold)


        print('RUNNING SIM ON VALID SET')
        valid_balance,threshold_ =  run_simulator(gbm_model,dvalid = dvalid,y_valid = y_valid,rr = tp / sl,train = False,threshold = threshold)

        print('RUNNING SIM ON 2022 SET')
        balance_2022,threshold_ =  run_simulator(gbm_model,dvalid = dtest2,y_valid = y_test2,rr = tp / sl,train = False,threshold = threshold)

                
    
    save_xgb_model(model = gbm_model,params = params,auc_score = score,auc_score2 = auc_train,
                   train_balance = train_balance,valid_balance = valid_balance,test_balance = test_balance,balance_2022 = balance_2022,threshold = threshold)
    
    
    auc_difference = abs(auc_train - auc_test)
    auc_difference2 = abs(auc_train - auc_valid)
    
    # Penalize the validation AUC by the difference between the training and validation AUC
    score = auc_train - (auc_difference * .3)  #(auc_difference2 * .25)##(auc_difference * .25)# - (auc_difference2 * .25)
    #score = auc_train

    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}


def optimize(
             trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 25, 150, 1),
         #'n_estimators': 10,
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 8, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        'scale_pos_weight': hp.uniform('scale_pos_weight', 1, int((y_train.shape[0] - y_train.sum()) / y_train.sum()) + 10),
    
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        #'tree_method': 'exact',
        'tree_method': 'hist',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                max_evals=100, trials=trials)
    return best




start = datetime(2005,1,1,0,0,0)
end = datetime(2023,1,1,0,0,0)


sl = 5
tp = 100
bullish = True
spread = .5


lst = [
    [5,100,True],
    [5,100,False],
    [5,200,True],
    [5,200,False],
    [10,100,True],
    [10,100,False],
    [10,200,True],
    [10,200,False],
    [10,500,True],
    [10,500,False]
    
    
    
]


lst = [
    [10,500,True],
    [10,200,True],
    [10,100,True],
    [5,200,True],
    [5,100,True]


    
    
    
]

lst = [
    [5,200,True],
    [5,100,True],
    [10,200,True],
    [10,100,True]
        
]
for l in lst:
    print(l)
    sl = l[0]
    tp = l[1]
    bullish = l[2]


    X_train, X_valid, y_train, y_valid,X_test, X_test2, y_test, y_test2 = load_train_and_test_data(start,end,sl,tp,bullish,spread)

    encoder = LabelEncoder()
    
    feature_types = get_feature_types(X_train)
    col = 'pair'
    encoder.fit(X_train[col])
    
    directory_path = os.getcwd() + "/ML_MODEL_REGULARIZE5_BULLISH_" + str(bullish) + "_TP_" + str(tp) + "_SL_" + str(sl) + "/"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    with open(directory_path + str(date.today()) + "_encoder.pkl", "wb") as f:
        pickle.dump(encoder, f)
        
    
    X_train[col] = encoder.transform(X_train[col])
    X_valid[col] = encoder.transform(X_valid[col])
    X_test[col] = encoder.transform(X_test[col])
    X_test2[col] = encoder.transform(X_test2[col])

    X_train.head()


    trials = Trials()
    try:
        best_hyperparams = optimize(
                                trials
                                )
    except:
        print('FAILLURE')
    print("The best hyperparameters are: ", "\n")
    print(best_hyperparams)



final balance                                                                        
10000000                                                                             
threshold                                                                            
0.65                                                                                 
RUNNING SIM ON TEST SET                                                              
final balance                                                                        
41055                                                                                
threshold                                                                            
0.65                                                                                 
RUNNING SIM ON VALID SET                                                             
final balance                                                                        
18341                                                 

# best model
## ML_MODEL_REGULARIZE4_BULLISH_True_TP_200_SL_5
## AUC_train_84.68_AUC_test_83.9_SIM_BALANCE_TRAIN_10000000_VALID_36908_TEST_10000000_2022_109258_threshold_0.78


In [ ]:
X_test.shape

In [ ]:
import re
# Function to extract the AUC_test score from the file name
def get_auc_test_score(file_name):
    match = re.search(r"AUC_test_([\d.]+)", file_name)
    if match:
        return float(match.group(1))
    return 0
def load_xgb_model(model_path):

    print('LOADING XGB MODEL FROM:',model_path)
    model = pickle.load(open(model_path, "rb"))
    return model

folder_path = os.getcwd() + '/ML_MODEL_V2/'  # Replace with your folder path
file_pattern = 'SIM_BALANCE'
# List all files in the folder and filter based on the desired string
files = [file_name for file_name in os.listdir(folder_path) if file_pattern in file_name]

# Sort the files based on the extracted AUC_test score
sorted_files = sorted(files, key=get_auc_test_score)


# Print the sorted file names
for folder_name in reversed(sorted_files):
    print()
    print()
    print()
    print(f"File: {folder_name}")
    model_path = os.getcwd() + '/ML_MODEL_V2/' + folder_name + '/xgb.pkl'
    gbm_model = load_xgb_model(model_path)
    s = 100000
    e = 200000
    for s in [0,100000,200000]:
        e = s + 100000
        print(s,e)
        dvalid = xgb.DMatrix(X_test.iloc[s:e,:], label=y_test.iloc[s:e])
        run_simulator(gbm_model,dvalid = dvalid,y_valid = y_test.iloc[s:e])
        

        print()
        
    print('RUNNING FINAL HOLDOUT SET 2022...')
    dvalid = xgb.DMatrix(X_test2, label=y_test2)
    run_simulator(gbm_model,dvalid = dvalid,y_valid = y_test2)
    

In [ ]:
model_path = '/Users/aidanmcconnell/Documents/MARKET_ANALYTICS/forex/Macro/ML_MODEL_V2/AUC_train_73.38_AUC_test_69.95_SIM_BALANCE_10000000/xgb.pkl'
gbm_model = load_xgb_model(model_path)

dvalid = xgb.DMatrix(X_test2, label=y_test2)
run_simulator(gbm_model,dvalid = dvalid,y_valid = y_test2)
print()